## Save intermediate feature-maps to file

This short notebook performs a forward-pass on a single ImageNet image, and save the intermediate results to a file for later inspection.

In [ ]:
import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable

# Relative import of code from distiller, w/o installing the package
import os
import sys
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import distiller
import apputils
import models
from apputils import *

In [ ]:
# Load ResNet50
resnet50 = models.create_model(pretrained=True, dataset='imagenet', arch='resnet50', parallel=False)

# Load the "sparse" compressed model 
resnet50_sparse = models.create_model(pretrained=True, dataset='imagenet', arch='resnet50', parallel=True)
load_checkpoint(resnet50_sparse , "../examples/classifier_compression/resnet50_checkpoint_70.66-sparse_76.09-top1.pth.tar");
resnet50_sparse  = distiller.make_non_parallel_copy(resnet50_sparse)


In [ ]:
from torchvision import transforms

normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Resize(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   normalize
])

# Open some file and do a forward-pass.  We do this to check everything is cool.
img = Image.open("../../data.imagenet/val/n02669723/ILSVRC2012_val_00029372.JPEG")
img = preprocess(img)
img = img.unsqueeze_(0)

sample = Variable(img)

resnet50_sparse.eval()
predictions = resnet50_sparse(sample.cuda())

top1_vals, top1_indices = torch.max(predictions, 1)
print(top1_vals.item())

In [ ]:
# This is where the work gets done:
# We register for forward callbacks, perform a forward pass, and in the callbacks we cache the
# intermediate feature-maps.

intermediary_results = {}
cb_handles = []

def save_conv_output(m, i, o):
    intermediary_results[m.distiller_name] = o.data.cpu().numpy()
    
def register_forward_hook(m):
    if hasattr(m, 'distiller_name'):
        h = m.register_forward_hook(save_conv_output)
        cb_handles.append(h)

distiller.utils.assign_layer_fq_names(resnet50_sparse)
resnet50_sparse.apply(register_forward_hook)
predictions = resnet50_sparse(sample.cuda())
for h in cb_handles:
    h.remove()
print(len(intermediary_results))

In [ ]:
# Now save the feature-maps to file...
import pickle 

with open('resnet50_sparse_fms.pickle', 'wb') as f:
    pickle.dump(intermediary_results, f)
    #np_weights = pickle.save(handle)

In [ ]:
# This is how you load results later...

intermediary_results_from_file = {}

with open('resnet50_sparse_fms.pickle', 'rb') as f:
    intermediary_results_from_file = pickle.load(f)
 
# To print all layer names:
for name, tensor in intermediary_results_from_file.items():
    print(name, tensor.size)